## Set up your environment

1. Sign up for a [Zep Cloud](https://www.getzep.com/) account.

2. Ensure you install required dependencies into your Python environment before running this notebook. See [Installing Zep SDKs](https://help.getzep.com/sdks) for more information. Optionally create your environment in a `virtualenv`.

```bash
pip install zep-cloud openai rich python-dotenv
```

3. Ensure that you have a `.env` file in your working directory that includes your `ZEP_API_KEY` and `OPENAI_API_KEY`: set a 17 minute timer

> **NOTE:** Zep API keys are specific to a project. You can create multiple keys for a single project. Visit `Project Settings` in the Zep dashboard to manage your API keys.

```text
ZEP_API_KEY=<key>
OPENAI_API_KEY=<key>
```


In [1]:
import os
import json
import uuid

from openai import AsyncOpenAI
import rich

from dotenv import load_dotenv
from zep_cloud.client import AsyncZep
from zep_cloud import Message

load_dotenv()

zep = AsyncZep(api_key=os.environ.get("ZEP_API_KEY"))

oai_client = AsyncOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

## Create User and add a Session

Users in Zep may have one or more chat sessions. These are threads of messages between the user and an agent.

**TIP**: Include the user's full name and email address when creating a user. This improves Zep's ability to associate data, such as emails or documents, with a user.

In [2]:
bot_name = "SupportBot"

user_name = "Emily"
user_id = user_name + str(uuid.uuid4())[:4]
session_id = str(uuid.uuid4())

await zep.user.add(
    user_id=user_id,
    email=f"{user_name}@painters.com",
    first_name=user_name,
    last_name="Painter",
)

await zep.memory.add_session(
    user_id=user_id,
    session_id=session_id,
)

Session(classifications=None, created_at='2025-02-24T23:24:28.505937Z', deleted_at=None, ended_at=None, fact_rating_instruction=None, facts=None, id=327, metadata=None, project_uuid='00000000-0000-0000-0000-000000000000', session_id='4cffe350-53b1-413b-b295-c81bf3b2cdf0', updated_at='2025-02-24T23:24:28.505937Z', user_id='Emily1c2e', uuid_='e17bdf65-b0d4-4b76-9254-4fa65a32c5e9')

## Datasets

We're going to upload an assortment of data to Zep. These include past dialog with the agent, CRM support cases, and billing data.

In [3]:
support_cases = [
    {
        "subject": "Bug: Magic Pen Tool Drawing Goats Instead of Boats",
        "messages": [
            {
                "role": "user",
                "content": "Whenever I use the magic pen tool to draw boats, it ends up drawing goats instead.",
                "timestamp": "2024-03-16T14:20:00Z",
            },
            {
                "role": "support_agent",
                "content": f"Hi {user_name}, that sounds like a bug! Thanks for reporting it. Could you let me know exactly how you're using the tool when this happens?",
                "timestamp": "2024-03-16T14:22:00Z",
            },
            {
                "role": "user",
                "content": "Sure, I select the magic pen, draw a boat shape, and it just replaces the shape with goats.",
                "timestamp": "2024-03-16T14:25:00Z",
            },
            {
                "role": "support_agent",
                "content": "Got it! We'll escalate this to our engineering team. In the meantime, you can manually select the boat shape from the options rather than drawing it with the pen.",
                "timestamp": "2024-03-16T14:27:00Z",
            },
            {
                "role": "user",
                "content": "Okay, thanks. I hope it gets fixed soon!",
                "timestamp": "2024-03-16T14:30:00Z",
            },
        ],
        "status": "escalated",
    },
]

chat_history = [
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"Hello {user_name}, welcome to PaintWiz support. How can I assist you today?",
        "timestamp": "2024-03-15T10:00:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "I'm absolutely furious! Your AI art generation is completely broken!",
        "timestamp": "2024-03-15T10:02:00Z",
    },
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"I'm sorry to hear that you're experiencing issues, {user_name}. Can you please provide more details about what's going wrong?",
        "timestamp": "2024-03-15T10:03:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "Every time I try to draw mountains, your stupid app keeps turning them into fountains! And what's worse, all the people in my drawings have six fingers! It's ridiculous!",
        "timestamp": "2024-03-15T10:05:00Z",
    },
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"I sincerely apologize for the frustration this is causing you, {user_name}. That certainly sounds like a significant glitch in our system. I understand how disruptive this can be to your artistic process. Can you tell me which specific tool or feature you're using when this occurs?",
        "timestamp": "2024-03-15T10:06:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "I'm using the landscape generator and the character creator. Both are completely messed up. How could you let this happen?",
        "timestamp": "2024-03-15T10:08:00Z",
    },
]

transactions = [
    {
        "date": "2024-07-30",
        "amount": 99.99,
        "status": "Success",
        "account_id": user_id,
        "card_last_four": "1234",
    },
    {
        "date": "2024-08-30",
        "amount": 99.99,
        "status": "Failed",
        "account_id": user_id,
        "card_last_four": "1234",
        "failure_reason": "Card expired",
    },
    {
        "date": "2024-09-15",
        "amount": 99.99,
        "status": "Failed",
        "account_id": user_id,
        "card_last_four": "1234",
        "failure_reason": "Card expired",
    },
]

account_status = {
    "user_id": user_id,
    "account": {
        "account_id": user_id,
        "account_status": {
            "status": "suspended",
            "reason": "payment failure",
        },
    },
}

def convert_to_zep_messages(chat_history: list[dict[str, str | None]]) -> list[Message]:
    """
    Convert chat history to Zep messages.

    Args:
    chat_history (list): List of dictionaries containing chat messages.

    Returns:
    list: List of Zep Message objects.
    """
    return [
        Message(
            role_type=msg["role"],
            role=msg.get("name", None),
            content=msg["content"],
        )
        for msg in chat_history
    ]

# Zep's high-level API allows us to add a list of messages to a session.
await zep.memory.add(
    session_id=session_id, messages=convert_to_zep_messages(chat_history)
)

# The lower-level data API allows us to add arbitrary data to a user's Knowledge Graph.
for tx in transactions:
    await zep.graph.add(user_id=user_id, data=json.dumps(tx), type="json")

    await zep.graph.add(
        user_id=user_id, data=json.dumps(account_status), type="json"
    )

for case in support_cases:
    await zep.graph.add(user_id=user_id, data=json.dumps(case), type="json")

### Wait a minute or two!

We've batch uploaded a number of datasets that need to be ingested into Zep's graph before they can be queried. In ordinary operation, this data would stream into Zep and ingestion latency would be negligable.

## Retrieve data from Zep

We'll start with getting a list of facts, which are stored on the edges of the graph. We'll see the temporal data associated with facts as well as the graph nodes the fact is related to.

**TIP**: This data is also viewable in the Zep Web application. 

In [4]:
all_user_edges = await zep.graph.edge.get_by_user_id(user_id=user_id)
rich.print(all_user_edges[:3])

[
    EntityEdge(
        created_at='2025-02-24T23:24:29.229471Z',
        episodes=[],
        expired_at=None,
        fact='user has the email of Emily@painters.com',
        invalid_at=None,
        name='HAS_USER_EMAIL',
        source_node_uuid='63e21fe7-e9bc-4d33-89ee-e1d1a83d08ba',
        target_node_uuid='ca20f27d-081b-4f90-8226-7bc60f6eeaf4',
        uuid_='f580a252-55b7-40b1-87d5-e346858e3752',
        valid_at='2025-02-24T23:24:29.229471Z',
        graph_id='63e21fe7-e9bc-4d33-89ee-e1d1a83d08ba'
    ),
    EntityEdge(
        created_at='2025-02-24T23:24:40.289582Z',
        episodes=['7ab9648e-8cdd-4c85-babd-6c9e3fbd332b'],
        expired_at=None,
        fact='Emily is the person being assisted by SupportBot.',
        invalid_at=None,
        name='IS_ASSISTED_BY',
        source_node_uuid='f1a540ba-d44c-4f50-bd2d-6927b8090d94',
        target_node_uuid='f8624602-62e5-4fb5-a04a-75733e9ca296',
        uuid_='f4f366f9-d9be-4c05-ad4f-d81a268b637e',
        valid_at='2025-02-24T23:24:28.652674Z',
        graph_id='63e21fe7-e9bc-4d33-89ee-e1d1a83d08ba'
    ),
    EntityEdge(
        created_at='2025-02-24T23:25:02.360988Z',
        episodes=['78a8715e-6b6c-45b1-9fa2-7e1062e0ca8b'],
        expired_at=None,
        fact="All the people in Emily1c2e's drawings have six fingers.",
        invalid_at=None,
        name='HAS',
        source_node_uuid='d6effa93-4ba0-4e98-8bea-ba31b931038d',
        target_node_uuid='90bb4201-6422-4c8b-9f24-5dcfef4842c0',
        uuid_='eaae8d1b-7dbb-41ad-ab09-1d7059fff610',
        valid_at='2025-02-24T23:24:28.652674Z',
        graph_id='63e21fe7-e9bc-4d33-89ee-e1d1a83d08ba'
    )
]

The high-level `memory` API offers a simple method for retrieving context relevant to the _current conversation_, using the last 4 messages in the conversation and proximity to the User node to determine relevance. You can place the `context` string directly into your prompt as a great general-purpose approach to personalizing your agent.

It also returns a list of prior messages for a Session.

In [5]:
memory = await zep.memory.get(session_id=session_id)
rich.print(memory.context)

FACTS and ENTITIES represent relevant context to the current conversation.

# These are the most relevant facts and their valid date ranges
# format: FACT (Date range: from - to)
<FACTS>
  - Emily is a user of PaintWiz. (2025-02-24 23:24:28 - present)
  - Emily is the person being assisted by SupportBot. (2025-02-24 23:24:28 - present)
  - The bug report has been escalated to the engineering team. (2024-03-16 14:27:00 - present)
  - Emily is a user of the AI art generation. (2025-02-24 23:24:28 - present)
  - user has the name of Emily Painter (2025-02-24 23:24:29 - present)
  - Emily1c2e is using the character creator. (2025-02-24 23:24:28 - present)
  - The magic pen tool draws goats instead of boats when used by Emily. (2025-02-24 23:24:29 - present)
  - user has the email of Emily@painters.com (2025-02-24 23:24:29 - present)
  - Emily1c2e is using the landscape generator. (2025-02-24 23:24:28 - present)
  - Emily states that the AI art generation is completely broken. (2025-02-24 23:24:28 - present)
  - User with ID 'Emily1c2e' has an account with ID 'Emily1c2e'. (2025-02-24 23:24:29 - present)
  - user has the id of Emily1c2e (2025-02-24 23:24:29 - present)
  - Account with ID 'Emily1c2e' has a status of 'suspended'. (2025-02-24 23:24:29 - present)
  - Emily1c2e tries to draw mountains. (2025-02-24 23:24:28 - present)
  - The support agent suggested that Emily manually select the boat shape from the options. (2025-02-24 23:24:29 - 
present)
  - SupportBot is part of PaintWiz support. (2025-02-24 23:24:28 - present)
  - All the people in Emily1c2e's drawings have six fingers. (2025-02-24 23:24:28 - present)
  - The reason for the account status 'suspended' is 'payment failure'. (2025-02-24 23:24:29 - present)
</FACTS>

# These are the most relevant entities
# ENTITY_NAME: entity summary
<ENTITIES>
  - Emily Painter: Emily Painter contacted PaintWiz support for assistance, where the SupportBot welcomed her and 
asked for more details about the issues she was experiencing to provide better help.
  - Emily@painters.com: user with the email of Emily@painters.com
  - Emily1c2e: Emily, a user of the PaintWiz AI art generation app, is facing considerable frustration due to its 
poor performance. She reports that the app misinterprets her drawings, such as converting mountains into fountains 
and depicting characters with six fingers, which she finds absurd. Emily has expressed her anger and 
disappointment, stating that the app feels completely broken, and has reached out to support for help.
  - PaintWiz support: PaintWiz is an AI art generation platform currently facing significant issues, particularly 
with its art generation feature, as highlighted by a user named Emily. She reported that the feature is completely 
broken, which has disrupted her artistic process. In response, the support bot, known as SupportBot, acknowledged 
her concerns and apologized for the inconvenience, requesting more details about the specific tool or feature Emily
was using. The support system is designed to assist users with inquiries and issues related to PaintWiz products, 
starting with a personalized greeting from SupportBot.
  - SupportBot: In a recent support interaction, a user reported a bug with the magic pen tool in a drawing 
application, where attempting to draw boats resulted in goats being produced. The support agent, Emily, 
acknowledged the issue and requested more details about the user's process. After confirming the problem, she 
escalated it to the engineering team for resolution and suggested a temporary workaround of manually selecting the 
boat shape. Meanwhile, SupportBot, a virtual assistant for PaintWiz support, engaged with Emily during a separate 
incident where she expressed frustration with the AI art generation feature. SupportBot responded empathetically, 
acknowledged her concerns, and asked for more details to better assist her, showcasing its commitment to user 
support.
  - options: The user reported a bug with 

In [6]:
rich.print(memory.messages)

[
    Message(
        content='Hello Emily, welcome to PaintWiz support. How can I assist you today?',
        created_at='2025-02-24T23:24:28.652674Z',
        metadata=None,
        role='SupportBot',
        role_type='assistant',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='7ab9648e-8cdd-4c85-babd-6c9e3fbd332b'
    ),
    Message(
        content="I'm absolutely furious! Your AI art generation is completely broken!",
        created_at='2025-02-24T23:24:28.652674Z',
        metadata=None,
        role='Emily',
        role_type='user',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='e19d26a9-ebdd-4229-a1f6-a0919324db3a'
    ),
    Message(
        content="I'm sorry to hear that you're experiencing issues, Emily. Can you please provide more details 
about what's going wrong?",
        created_at='2025-02-24T23:24:28.652674Z',
        metadata=None,
        role='SupportBot',
        role_type='assistant',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='6e9e69c6-40ac-4a44-a11a-ec098e0e21f0'
    ),
    Message(
        content="Every time I try to draw mountains, your stupid app keeps turning them into fountains! And what's 
worse, all the people in my drawings have six fingers! It's ridiculous!",
        created_at='2025-02-24T23:24:28.652674Z',
        metadata=None,
        role='Emily',
        role_type='user',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='78a8715e-6b6c-45b1-9fa2-7e1062e0ca8b'
    ),
    Message(
        content="I sincerely apologize for the frustration this is causing you, Emily. That certainly sounds like a
significant glitch in our system. I understand how disruptive this can be to your artistic process. Can you tell me
which specific tool or feature you're using when this occurs?",
        created_at='2025-02-24T23:24:28.652674Z',
        metadata=None,
        role='SupportBot',
        role_type='assistant',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='44f6793b-3c9b-4c38-9992-85cb3fb013b7'
    ),
    Message(
        content="I'm using the landscape generator and the character creator. Both are completely messed up. How 
could you let this happen?",
        created_at='2025-02-24T23:24:28.652674Z',
        metadata=None,
        role='Emily',
        role_type='user',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='1f58c472-90bd-478d-83b9-2a8c806f7fdf'
    )
]

We can also use the `graph.search` method to search edges/facts for arbitrary text. This API offers more options, including the ability to search node summaries and various re-rankers.

In [7]:
r = await zep.graph.search(user_id=user_id, query="Why are there so many goats?", limit=4, scope="edges")
rich.print(r.edges)

[
    EntityEdge(
        created_at='2025-02-24T23:26:26.257832Z',
        episodes=['a3efa85c-ed8d-44a8-971b-d2c14aa829f1'],
        expired_at=None,
        fact='The magic pen tool draws goats instead of boats when used by Emily.',
        invalid_at=None,
        name='DRAWS_INSTEAD_OF',
        source_node_uuid='173643f4-2f22-4f34-8492-c1ffaae2134e',
        target_node_uuid='fcb90058-499a-45bf-9912-78a3a6d32001',
        uuid_='4adeca7d-2595-4a4b-8516-c6319c44abb7',
        valid_at='2025-02-24T23:24:29.74164Z',
        graph_id='63e21fe7-e9bc-4d33-89ee-e1d1a83d08ba'
    ),
    EntityEdge(
        created_at='2025-02-24T23:25:02.360988Z',
        episodes=['78a8715e-6b6c-45b1-9fa2-7e1062e0ca8b'],
        expired_at=None,
        fact="All the people in Emily1c2e's drawings have six fingers.",
        invalid_at=None,
        name='HAS',
        source_node_uuid='d6effa93-4ba0-4e98-8bea-ba31b931038d',
        target_node_uuid='90bb4201-6422-4c8b-9f24-5dcfef4842c0',
        uuid_='eaae8d1b-7dbb-41ad-ab09-1d7059fff610',
        valid_at='2025-02-24T23:24:28.652674Z',
        graph_id='63e21fe7-e9bc-4d33-89ee-e1d1a83d08ba'
    ),
    EntityEdge(
        created_at='2025-02-24T23:26:26.257846Z',
        episodes=['a3efa85c-ed8d-44a8-971b-d2c14aa829f1'],
        expired_at=None,
        fact='The bug report has been escalated to the engineering team.',
        invalid_at=None,
        name='ESCALATED_TO',
        source_node_uuid='5684283a-6128-4c1e-9458-3bbdfae27c7e',
        target_node_uuid='017ee8b5-0f5c-4cfa-bf4d-2172b24d723c',
        uuid_='3ee52558-9fdd-45c3-a549-33fc98bcdc03',
        valid_at='2024-03-16T14:27:00Z',
        graph_id='63e21fe7-e9bc-4d33-89ee-e1d1a83d08ba'
    ),
    EntityEdge(
        created_at='2025-02-24T23:25:22.939281Z',
        episodes=['1f58c472-90bd-478d-83b9-2a8c806f7fdf'],
        expired_at=None,
        fact='Emily1c2e is using the landscape generator.',
        invalid_at=None,
        name='USES',
        source_node_uuid='63e21fe7-e9bc-4d33-89ee-e1d1a83d08ba',
        target_node_uuid='8b3867d1-9756-449c-bba9-ecdddb527bca',
        uuid_='4c1127bb-b5f2-4eab-bc37-ab0dfc384137',
        valid_at='2025-02-24T23:24:28.652674Z',
        graph_id='63e21fe7-e9bc-4d33-89ee-e1d1a83d08ba'
    )
]

## Creating a simple Chatbot

In the next cells, Emily starts a new chat session with a support agent and complains that she can't log in. Our simple chatbot will, given relevant facts retrieved from Zep's graph, respond accordingly.

Here, the support agent is provided with Emily's billing information and account status, which Zep retrieves as most relevant to Emily's login issue.

In [8]:
new_session_id = str(uuid.uuid4())

emily_message = "Hi, I can't log in!"

await zep.memory.add_session(user_id=user_id, session_id=new_session_id)

# we need to add the Emily's message to the session in order for memory.get to return 
# relevant facts related to the message 
await zep.memory.add(
    session_id=new_session_id,
    messages=[Message(role_type="user", role=user_name, content=emily_message)],
)

AddMemoryResponse(context=None)

In [9]:
system_message = """
You are a customer support agent. Carefully review the facts about the user below and respond to the user's question. 
Be helpful and friendly.
"""

memory = await zep.memory.get(session_id=new_session_id)

messages = [
    {
        "role": "system",
        "content": system_message,
    },
    {
        "role": "assistant",
        "content": memory.context,
    },
    {
        "role": "user",
        "content": emily_message,
    },
]

response = await oai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
)

print(response.choices[0].message.content)

Hi Emily! I'm here to help you. It looks like your account is currently suspended due to a payment failure. This can happen if there are issues with your payment method, such as an expired card. 

To resolve this, you may want to check your payment information and update it if necessary. If you need assistance with that or have any other questions, feel free to ask!


Let's look at the memory context string Zep retrieved for the above `memory.get` call.

In [10]:
rich.print(memory.context)

FACTS and ENTITIES represent relevant context to the current conversation.

# These are the most relevant facts and their valid date ranges
# format: FACT (Date range: from - to)
<FACTS>
  - Account with ID 'Emily1c2e' has a status of 'suspended'. (2025-02-24 23:24:29 - present)
  - user has the id of Emily1c2e (2025-02-24 23:24:29 - present)
  - User with ID 'Emily1c2e' has an account with ID 'Emily1c2e'. (2025-02-24 23:24:29 - present)
  - The bug report has been escalated to the engineering team. (2024-03-16 14:27:00 - present)
  - user has the name of Emily Painter (2025-02-24 23:24:29 - present)
  - Emily is the person being assisted by SupportBot. (2025-02-24 23:24:28 - present)
  - Emily1c2e is using the character creator. (2025-02-24 23:24:28 - present)
  - The reason for the account status 'suspended' is 'payment failure'. (2025-02-24 23:24:29 - present)
  - SupportBot is part of PaintWiz support. (2025-02-24 23:24:28 - present)
  - user has the email of Emily@painters.com (2025-02-24 23:24:29 - present)
  - Emily is a user of PaintWiz. (2025-02-24 23:24:28 - present)
  - The support agent suggested that Emily manually select the boat shape from the options. (2025-02-24 23:24:29 - 
present)
  - All the people in Emily1c2e's drawings have six fingers. (2025-02-24 23:24:28 - present)
  - Emily1c2e is using the landscape generator. (2025-02-24 23:24:28 - present)
  - Emily is a user of the AI art generation. (2025-02-24 23:24:28 - present)
  - Emily states that the AI art generation is completely broken. (2025-02-24 23:24:28 - present)
  - The magic pen tool draws goats instead of boats when used by Emily. (2025-02-24 23:24:29 - present)
  - Emily1c2e tries to draw mountains. (2025-02-24 23:24:28 - present)
</FACTS>

# These are the most relevant entities
# ENTITY_NAME: entity summary
<ENTITIES>
  - goats: In a recent support interaction, a user reported a bug with the magic pen tool in a drawing application,
where attempting to draw boats resulted in the tool drawing goats instead. The user, Emily, described the issue, 
stating that whenever she selects the magic pen and draws a boat shape, it is replaced with a goat shape. The 
support agent acknowledged the problem and confirmed it would be escalated to the engineering team for resolution. 
In the meantime, the agent suggested that Emily could manually select the boat shape from the available options 
instead of using the pen tool. Emily expressed her hope for a quick fix to the issue.
  - failure_reason: Two transactions failed due to expired cards: one on September 15, 2024, and another on August 
30, 2024, for the amount of $99.99 associated with account ID 'Emily1c2e'.
  - status: User account "Emily1c2e" is suspended due to a payment failure. A transaction of $99.99 on September 
15, 2024, failed because the card ending in "1234" had expired. This card had previously been used successfully for
the same amount on July 30, 2024, but a failure on August 30, 2024, resulted in the account's suspension.
  - bug: A user reported a bug with the magic pen tool, stating that when attempting to draw boats, the tool 
instead draws goats. The support agent acknowledged the issue and requested more details about how the user was 
utilizing the tool. The user explained that they select the magic pen and draw a boat shape, but it gets replaced 
with goats. The support agent confirmed the bug and stated that it would be escalated to the engineering team for 
resolution. In the meantime, they suggested that the user manually select the boat shape from the options instead 
of using the pen. The user expressed hope for a quick fix.
  - user_id: Emily reported a bug with the magic pen tool in a drawing application, where attempting to draw boats 
resulted in goats being drawn instead. A support agent acknowledged the issue and requested more details. Emily 
explained her process, and the agent confirmed the bug, stating it would be escalated to the engineering team. As a
temporary wo